In [1]:
import pandas as pd
pd.set_option("display.float_format", "{:,.2f}".format)
from tabulate import tabulate
table_source = "../logs/smartRPA/202511-results/validation_experiment_results_word2vec_safety2_olap08_rho09.csv"

In [2]:
df = pd.read_csv(table_source)
df.rename(columns={"motif0-length":"motLength","motif0-occurances":"motOcc","motif0-shuffle":"motShuffle"}, inplace=True)

In [3]:
pivotPercentage = pd.pivot_table(df, index=["percentageMotifsOverLog"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotLogLength  = pd.pivot_table(df, index=["logLength"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotNoOfMotifs = pd.pivot_table(df, index=["noOfMotifs"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotMotOcc     = pd.pivot_table(df, index=["motOcc"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotMotLength  = pd.pivot_table(df, index=["motLength"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotMotShuffle = pd.pivot_table(df, index=["motShuffle"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')

list_of_pivots = [pivotPercentage, pivotNoOfMotifs, pivotMotOcc, pivotMotLength, pivotMotShuffle]

In [4]:
def calculate_classification_metrics(df):
    # Create a copy to avoid SettingWithCopy warnings if the input is a slice
    res_df = df.copy()

    # 1. Recall: TP / (TP + FN)
    # Measures: Out of all the actual motifs in the data, how many did we find?
    res_df["recall"] = res_df["total_tp"] / (res_df["total_tp"] + res_df["total_fn"])

    # 2. Precision: TP / (TP + FP)
    # Measures: Out of all the motifs we claimed to find, how many were actually correct?
    res_df["precision"] = res_df["total_tp"] / (res_df["total_tp"] + res_df["total_fp"])

    # 3. F1-score: Harmonic mean of Precision and Recall
    # Measures: Balance between Precision and Recall.
    res_df["f1-score"] = (2 * (res_df["precision"] * res_df["recall"]) / 
                          (res_df["precision"] + res_df["recall"]))

    # Optional: Fill NaN values with 0.0 (happens if denominator is 0)
    res_df[["recall", "precision", "f1-score"]] = res_df[["recall", "precision", "f1-score"]].fillna(0.0)

    return res_df

# Usage:
for pivot in list_of_pivots:
    pivot = calculate_classification_metrics(pivot)
    # Assuming 'final_results' is your DataFrame
    print(tabulate(pivot, headers='keys', tablefmt='psql', numalign="right"))

+---------------------------+------------+------------+------------+----------+-------------+------------+
|   percentageMotifsOverLog |   total_fn |   total_fp |   total_tp |   recall |   precision |   f1-score |
|---------------------------+------------+------------+------------+----------+-------------+------------|
|                         1 |          2 |      15653 |        658 |  0.99697 |   0.0403409 |   0.077544 |
|                        10 |         49 |       3239 |       2696 | 0.982149 |    0.454254 |   0.621198 |
|                        25 |        232 |        523 |       2738 | 0.921886 |     0.83962 |   0.878832 |
|                        50 |        623 |        440 |       2347 | 0.790236 |    0.842124 |   0.815355 |
|                        75 |        680 |        189 |       2290 | 0.771044 |     0.92376 |   0.840521 |
|                       100 |       2571 |        126 |        399 | 0.134343 |        0.76 |   0.228326 |
+---------------------------+--------

In [5]:
dfFilteredOne = df[~df["percentageMotifsOverLog"].isin([1.00,100.00])]
pivotPercentageFiltered = pd.pivot_table(dfFilteredOne, index=["percentageMotifsOverLog"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotLogLengthFiltered  = pd.pivot_table(dfFilteredOne, index=["logLength"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotNoOfMotifsFiltered = pd.pivot_table(dfFilteredOne, index=["noOfMotifs"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotMotOccFiltered     = pd.pivot_table(dfFilteredOne, index=["motOcc"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotMotLengthFiltered  = pd.pivot_table(dfFilteredOne, index=["motLength"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')
pivotMotShuffleFiltered = pd.pivot_table(dfFilteredOne, index=["motShuffle"], values=["total_tp","total_fp","total_fn"], aggfunc='sum')

list_of_filtered_pivots = [pivotPercentageFiltered, pivotNoOfMotifsFiltered, pivotMotOccFiltered, pivotMotLengthFiltered, pivotMotShuffleFiltered]

In [6]:
for pivot in list_of_filtered_pivots:
    pivot = calculate_classification_metrics(pivot)
    # Assuming 'final_results' is your DataFrame
    print(tabulate(pivot, headers='keys', tablefmt='psql', numalign="right"))

+---------------------------+------------+------------+------------+----------+-------------+------------+
|   percentageMotifsOverLog |   total_fn |   total_fp |   total_tp |   recall |   precision |   f1-score |
|---------------------------+------------+------------+------------+----------+-------------+------------|
|                        10 |         49 |       3239 |       2696 | 0.982149 |    0.454254 |   0.621198 |
|                        25 |        232 |        523 |       2738 | 0.921886 |     0.83962 |   0.878832 |
|                        50 |        623 |        440 |       2347 | 0.790236 |    0.842124 |   0.815355 |
|                        75 |        680 |        189 |       2290 | 0.771044 |     0.92376 |   0.840521 |
+---------------------------+------------+------------+------------+----------+-------------+------------+
+--------------+------------+------------+------------+----------+-------------+------------+
|   noOfMotifs |   total_fn |   total_fp |   total